In [1]:
import os
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import time
import plotly.express as px
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as smt

In [2]:
df=pd.read_csv('Combined_News_DJIA.csv')

In [3]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [4]:
df.tail()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
1984,2016-06-27,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,Poland 'shocked' by xenophobic abuse of Poles ...,"There will be no second referendum, cabinet ag...","Scotland welcome to join EU, Merkel ally says",Sterling dips below Friday's 31-year low amid ...,No negative news about South African President...,Surge in Hate Crimes in the U.K. Following U.K...,...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...
1985,2016-06-28,1,"2,500 Scientists To Australia: If You Want To ...","The personal details of 112,000 French police ...",S&amp;P cuts United Kingdom sovereign credit r...,Huge helium deposit found in Africa,CEO of the South African state broadcaster qui...,"Brexit cost investors $2 trillion, the worst o...",Hong Kong democracy activists call for return ...,Brexit: Iceland president says UK can join 'tr...,...,"US, Canada and Mexico pledge 50% of power from...",There is increasing evidence that Australia is...,"Richard Branson, the founder of Virgin Group, ...","37,000-yr-old skull from Borneo reveals surpri...",Palestinians stone Western Wall worshipers; po...,Jean-Claude Juncker asks Farage: Why are you h...,"""Romanians for Remainians"" offering a new home...",Brexit: Gibraltar in talks with Scotland to st...,8 Suicide Bombers Strike Lebanon,Mexico's security forces routinely use 'sexual...
1986,2016-06-29,1,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,UK must accept freedom of movement to access E...,Devastated: scientists too late to captive bre...,British Labor Party leader Jeremy Corbyn loses...,A Muslim Shop in the UK Was Just Firebombed Wh...,Mexican Authorities Sexually Torture Women in ...,UK shares and pound continue to recover,...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...
1987,2016-06-30,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says pollution and 'stupidity'...,Boris Johnson says he will not run for Tory pa...,Six gay men in Ivory Coast were abused and for...,Switzerland denies citizenship to Muslim immig...,Palestinian terrorist stabs israeli teen girl ...,Puerto Rico will default on $1 billion of debt...,Republic of Ireland fans to be awarded medal f...,...,Googles free wifi at Indian railway stations i...,Mounting evidence suggests 'hobbits' were wipe...,The men who carried out Tuesday's terror attac...,Calls to suspend Saudi Arabia from UN Human Ri...,More Than 100 Nobel Laureates Call Out Greenpe...,British pedophile sentenced to 85 years in US ...,"US permitted 1,200 offshore fracks in Gulf of ...",We will be swimming in ridicule - French beach...,UEFA says no minutes of silence for Istanbul v...,Law Enforcement Sources: Gun Used in Paris Ter...
1988,2016-07-01,1,A 117-year-old woman in Mexico City finally re...,IMF chief backs Athens as permanent Olympic host,"The president of France says if Brexit won, so...",British Man Who Must Give Police 24 Hours' Not...,100+ Nobel laureates 

In [5]:
train=df[df['Date']<'20150101']
test=df[df['Date']>'20141231']

In [6]:
# Removing the punctuation
data=train.iloc[:,2:27]   #selecting column from 3rd to 27th
data.replace("[^a-zA-Z]"," ",regex=True,inplace=True) #replacing  everything except "a to z and A to Z" with space

In [7]:
#renaming the column names 

list1=[i for i in range(25)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,b Georgia downs two Russian warplanes as cou...,b BREAKING Musharraf to be impeached,b Russia Today Columns of troops roll into So...,b Russian tanks are moving towards the capital...,b Afghan children raped with impunity U N ...,b Russian tanks have entered South Ossetia...,b Breaking Georgia invades South Ossetia Rus...,b The enemy combatent trials are nothing but...,b Georgian troops retreat from S Osettain cap...,b Did the U S Prep Georgia for War with Russia,...,b Georgia Invades South Ossetia if Russia ge...,b Al Qaeda Faces Islamist Backlash,b Condoleezza Rice The US would not act to p...,b This is a busy day The European Union has ...,b Georgia will withdraw soldiers from Ir...,b Why the Pentagon Thinks Attacking Iran is a ...,b Caucasus in crisis Georgia invades South Os...,b Indian shoe manufactory And again in a se...,b Visitors Suffering from Mental Illnesses Ban...,b No Help for Mexico s Kidnapping Surge
1,b Why wont America and Nato help us If they w...,b Bush puts foot down on Georgian conflict,b Jewish Georgian minister Thanks to Israeli ...,b Georgian army flees in disarray as Russians ...,b Olympic opening ceremony fireworks faked,b What were the Mossad with fraudulent New Zea...,b Russia angered by Israeli military sale to G...,b An American citizen living in S Ossetia blam...,b Welcome To World War IV Now In High Definit...,b Georgia s move a mistake of monumental prop...,...,b Israel and the US behind the Georgian aggres...,b Do not believe TV neither Russian nor Geor...,b Riots are still going on in Montreal Canada...,b China to overtake US as largest manufacturer,b War in South Ossetia PICS,b Israeli Physicians Group Condemns State Tort...,b Russia has just beaten the United States ov...,b Perhaps the question about the Georgia R...,b Russia is so much better at war,b So this is what it s come to trading sex fo...


In [8]:
for index in new_Index:
    data[index]=data[index].str.lower() #converting all words into Lower

data.head(1)  

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,b georgia downs two russian warplanes as cou...,b breaking musharraf to be impeached,b russia today columns of troops roll into so...,b russian tanks are moving towards the capital...,b afghan children raped with impunity u n ...,b russian tanks have entered south ossetia...,b breaking georgia invades south ossetia rus...,b the enemy combatent trials are nothing but...,b georgian troops retreat from s osettain cap...,b did the u s prep georgia for war with russia,...,b georgia invades south ossetia if russia ge...,b al qaeda faces islamist backlash,b condoleezza rice the us would not act to p...,b this is a busy day the european union has ...,b georgia will withdraw soldiers from ir...,b why the pentagon thinks attacking iran is a ...,b caucasus in crisis georgia invades south os...,b indian shoe manufactory and again in a se...,b visitors suffering from mental illnesses ban...,b no help for mexico s kidnapping surge


In [9]:
' '.join(str(x) for x in data.iloc[1,0:25])

'b why wont america and nato help us  if they wont help us now  why did we help them in iraq   b bush puts foot down on georgian conflict  b jewish georgian minister  thanks to israeli training  we re fending off russia   b georgian army flees in disarray as russians advance   gori abandoned to russia without a shot fired  b olympic opening ceremony fireworks  faked   b what were the mossad with fraudulent new zealand passports doing in iraq   b russia angered by israeli military sale to georgia  b an american citizen living in s ossetia blames u s  and georgian leaders for the genocide of innocent people  b welcome to world war iv  now in high definition   b georgia s move  a mistake of monumental proportions   b russia presses deeper into georgia  u s  says regime change is goal  b abhinav bindra wins first ever individual olympic gold medal for india  b  u s  ship heads for arctic to define territory  b drivers in a jerusalem taxi station threaten to quit rather than work for their 

In [10]:
headlines =[]
for row in range(0,len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))

In [11]:
headlines[0]

'b georgia  downs two russian warplanes  as countries move to brink of war  b breaking  musharraf to be impeached   b russia today  columns of troops roll into south ossetia  footage from fighting  youtube   b russian tanks are moving towards the capital of south ossetia  which has reportedly been completely destroyed by georgian artillery fire  b afghan children raped with  impunity   u n  official says   this is sick  a three year old was raped and they do nothing  b     russian tanks have entered south ossetia whilst georgia shoots down two russian jets   b breaking  georgia invades south ossetia  russia warned it would intervene on so s side  b the  enemy combatent  trials are nothing but a sham  salim haman has been sentenced to       years  but will be kept longer anyway just because they feel like it   b georgian troops retreat from s  osettain capital  presumably leaving several hundred people killed   video   b did the u s  prep georgia for war with russia   b rice gives green

In [12]:
#countvectorizer ------------->(it is use to convert sentence into vector)[it helps ous to create bag of words]
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [13]:
#implement bag of word
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

In [14]:
traindataset[0]

<1x403977 sparse matrix of type '<class 'numpy.int64'>'
	with 342 stored elements in Compressed Sparse Row format>

In [15]:
#apply Randomforest
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')

randomclassifier.fit(traindataset,train['Label'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [16]:
traindataset.shape[0]

1863

In [17]:
##predict for the test dataset

test_transform=[]
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
    
test_dataset =countvector.transform(test_transform)    

prediction =randomclassifier.predict(test_dataset)

In [18]:
test.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
1611,2015-01-02,1,Most cases of cancer are the result of sheer b...,Iran dismissed United States efforts to fight ...,Poll: One in 8 Germans would join anti-Muslim ...,UK royal family's Prince Andrew named in US la...,Some 40 asylum-seekers refused to leave the bu...,Pakistani boat blows self up after India navy ...,Sweden hit by third mosque arson attack in a week,940 cars set alight during French New Year,...,Ukrainian minister threatens TV channel with c...,Palestinian President Mahmoud Abbas has entere...,Israeli security center publishes names of 50 ...,The year 2014 was the deadliest year yet in Sy...,A Secret underground complex built by the Nazi...,Restrictions on Web Freedom a Major Global Iss...,Austrian journalist Erich Mchel delivered a pr...,Thousands of Ukraine nationalists march in Kiev,Chinas New Years Resolution: No More Harvestin...,Authorities Pull Plug on Russia's Last Politic...
1612,2015-01-05,0,Moscow-&gt;Beijing high speed train will reduc...,Two ancient tombs were discovered in Egypt on ...,China complains to Pyongyang after N Korean so...,Scotland Headed Towards Being Fossil Fuel-Free...,Prime Minister Shinzo Abe said Monday he will ...,Sex slave at centre of Prince Andrew scandal f...,Gay relative of Hamas founder faces deportatio...,The number of female drug addicts in Iran has ...,...,The Islamic State has approved a 2015 budget o...,"Iceland To Withdraw EU Application, Lift Capit...",Blackfield Capital Founder Goes Missing: The v...,Rocket stage crashes back to Earth in rural Ch...,2 Dead as Aircraft Bombs Greek Tanker in Libya...,Belgian murderer Frank Van Den Bleeken to die ...,Czech President criticizes Ukrainian PM; says ...,3 Vietnamese jets join search for 16 missing F...,France seeks end to Russia sanctions over Ukraine,China scraps rare earths caps


In [19]:
test.loc[1863,:]

Date                                            2016-01-04
Label                                                    0
Top1     Saudi Arabia breaks off ties with Iran - Amid ...
Top2     Exclusive: UK Government urged to reveal its r...
Top3                 China halts trading as stocks fall 7%
Top4      Earthquake measuring 6.8 magnitude strikes India
Top5     A virus linked to shrinking newborns brains is...
Top6     New photos of SpaceX booster show sooty, but u...
Top7     Indian consulate in Afghanistan under attack b...
Top8                             Iran calls off Major Hajj
Top9     Denmark responds to Swedish border checks with...
Top10    David Cameron criticised for turning 'blind ey...
Top11    Denmark introduces temporary controls along Ge...
Top12    Israel Retaliates to Hezbollah bombing at Leba...
Top13    China Aims to Build 40 Nuclear Power Plants in...
Top14    According to obtained documents, after rape an...
Top15    Western Australian rooftop solar systems produ.

In [20]:
prediction

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [21]:
#accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [22]:
matrix=confusion_matrix(test['Label'],prediction)
print(matrix)

score=accuracy_score(test['Label'],prediction)
print(score)

report=classification_report(test['Label'],prediction)
print(report)

[[131  55]
 [  3 189]]
0.8465608465608465
              precision    recall  f1-score   support

           0       0.98      0.70      0.82       186
           1       0.77      0.98      0.87       192

    accuracy                           0.85       378
   macro avg       0.88      0.84      0.84       378
weighted avg       0.87      0.85      0.84       378



In [23]:
#now use TF-idf in place of countvectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier

In [24]:
tfidfvector=TfidfVectorizer(ngram_range=(2,2))
traindataset=tfidfvector.fit_transform(headlines)

In [25]:
#apply Randomforest
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')

randomclassifier.fit(traindataset,train['Label'])
label_ll = train['Label']

In [26]:
##predict for the test dataset

test_transform=[]
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
    
test_dataset =tfidfvector.transform(test_transform)    

prediction =randomclassifier.predict(test_dataset)

In [27]:
prediction

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [28]:
matrix=confusion_matrix(test['Label'],prediction)
print(matrix)

score=accuracy_score(test['Label'],prediction)
print(score)

report=classification_report(test['Label'],prediction)
print(report)

[[145  41]
 [ 21 171]]
0.8359788359788359
              precision    recall  f1-score   support

           0       0.87      0.78      0.82       186
           1       0.81      0.89      0.85       192

    accuracy                           0.84       378
   macro avg       0.84      0.84      0.84       378
weighted avg       0.84      0.84      0.84       378



In [29]:
traindataset

<1863x403977 sparse matrix of type '<class 'numpy.float64'>'
	with 771733 stored elements in Compressed Sparse Row format>

In [29]:
features = pd.DataFrame.sparse.from_spmatrix(traindataset)
label = pd.DataFrame(train['Label'])

In [30]:
train_dfs = pd.merge(features, label, left_index=True, right_index=True)

In [31]:
#This funcn creates batches
def perfect_batches2(df, batch_size):
    batched_data = []
    if len(df)%batch_size != 0:
        print('Cannot create a perfect batch size with the input. \nTry the following: ')
        for i in range(len(df)):
            if i != 0 and len(df)%i == 0:
                print('This number can do: ', i)
        print('\n\nEnd of numbers!')
        return None
    i = 0
    while i <= len(df)-batch_size+1:
        batch = []
        j = 0
        while j <= batch_size:
            #print('debug mode: ', df.values[i]) #This line is just for debugging
            #batch.append(torch.tensor(df.values[i])) #original
            batch.append(df.values[i]) #new --modified
            i += 1
            j += 1
        #batched_data.append(torch.stack(batch)) #original
        batched_data.append(np.stack(batch)) #new --modified
    #return torch.stack(batched_data).float()
    return np.stack(batched_data)#.float()

In [32]:
batched_train = perfect_batches2(train_dfs, 50)
#batched_test = perfect_batches2(test_df, 50)

NameError: name 'train_dfs' is not defined

In [33]:
import torch
from torchvision import datasets
from torchvision import transforms
# Tensor transform
transform = transforms.ToTensor()

# SVHN training datasets
#svhn_train = datasets.SVHN(root='data/', split='train', download=True, transform=transform)

batch_size = 21
num_workers = 0

# build DataLoaders for SVHN dataset
train_loader = torch.utils.data.DataLoader(dataset=train_dfs.values, #dff.values,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=num_workers)

KeyboardInterrupt: 

In [53]:
import torch
import torch.nn as nn

class BinaryClassifier(nn.Module):
    def __init__(self, input_len):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_len, 200)
        self.bn1 = nn.BatchNorm1d(200)
        self.fc2 = nn.Linear(200, 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 50)
        self.bn3 = nn.BatchNorm1d(50)
        self.fc4 = nn.Linear(50, 1)
        self.sigmoid = nn.Sigmoid()
        #self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.30)

    def forward(self, x):
        #x = x.double()
        x = self.fc1(x)
        x = self.bn1(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.bn3(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        #x = self.tanh(x)
        return x

In [31]:
traindataset.shape[1]

403977

In [32]:
import torch.optim as optim
def modelAndOptim(input_len, lr):
    #modell = movieModel(input_len)
    modell = BinaryClassifier(input_len)
    optimizer = optim.SGD(modell.parameters(), lr)
    #optimizer = optim.Adam(modell.parameters(), lr)
    loss_func = nn.BCEWithLogitsLoss()
    #loss_func = nn.BCELoss()
    #loss_func = nn.CrossEntropyLoss()
    return modell, optimizer, loss_func

In [33]:
def accuracy(pred, actual):
    pred = torch.tensor([1.0 if i >= 0.5 else 0.0 for i in pred])
    #print('preddd: ', pred)
    #actual = torch.tensor([1 if i >= 7.0 else 0 for i in actual])
    same = []
    for i in range(len(pred)):
        if pred[i] == actual[i]:
            same.append(1)
        else:
            same.append(0)
    same = torch.tensor(same)
    #print(pred)
    #print(actual)
    #print(same)
    return (sum(same)/len(same)).item()
    
#accuracy(v, a)

In [88]:
import sklearn
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
pca = PCA(n_components=100)
X_pca = pca.fit_transform(t_data)

In [78]:
t_data = traindataset.toarray()

In [87]:
t_data =  torch.tensor(t_data)

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6020873208 bytes.

In [69]:
#create a train method and validation
#Theres an error i spotted i have to make sure that it loops through the batch one at a time not all at once
model, optimizer, loss = modelAndOptim(traindataset.shape[1], 0.05)
def train(epoch, train_data, model, optimizer, loss):
    model.train()
    #model = model.double()
    sum_loss = []
    accuracy_count = []
    t_data = traindataset.toarray()
    t_data =  torch.tensor(t_data)
    l_data = label_ll
    l_data = torch.tensor(l_data)
    for e in range(epoch):
        print('Started', e)
        losses = []
        #acc = []
        acc = None
        
        #train_data = next(iter(train_data)).type(torch.float32)
        #print('train_data')
        #print(train_data)
        #print(train_data[0, :-1])
        #print(train_data[0, :])
        #for i in range(5):
        #for rec in batch:
        #print(train_data)
        #features = train_data[:, :-1]#, :-1]
        features = t_data
        #print(features)
        #break
        #label = train_data[:, -1:]
        label = l_data
        print(label)
        #break
        optimizer.zero_grad()

        predicted = model(features)
        #print('predited')
        #print('p: ', len(predicted))
        #print('l: ', label)
        predicted = predicted.view(len(predicted), 1)
        label = label.view(len(label), 1)
        #print('pridicted size: ', predicted.shape)
        #print('label dType: ', label.shape)
        #print('pridicted: ', predicted)
        #print('label: ', label)
        loss_error = loss(predicted.double(), label.double())
        losses.append(loss_error.item())
        loss_error.backward()
        optimizer.step()
        #acc.append(accuracy(predicted.view(-1), label.view(-1)))
        acc = accuracy(predicted.view(-1), label.view(-1))
        #print('Average error at current  ', sum(losses)/len(losses))
        #print('done')
        #print(predicted)
        print('Average error at ',e,' epoch:  ', sum(losses)/len(losses))
        #acr = sum(acc)/len(acc)
        acr = acc
        print('Accuracy: ', acr)
        accuracy_count.append(acr)
        sum_loss.append(sum(losses)/len(losses))
        if acr >= 0.999:
            plt.plot(sum_loss, label = 'loss')
            #print(sum_loss)
            plt.legend()
            plt.show()
            return predicted, label
    #print('\nOverall Accuracy: ', sum(accuracy_count)/len(accuracy_count))
    #print('\n\n\n\n')
    #plt.plot(label, label = 'True')
    #plt.plot(sum_loss, label = 'loss')
    #print(sum_loss)
    #plt.legend()
    #plt.show()
    print('end of train')
    print("")
    return predicted, label

In [ ]:
def train(epoch, train_data, model, optimizer, loss):
    model.train()
    #model = model.double()
    sum_loss = []
    accuracy_count = []
    t_data = traindataset.toarray()
    t_data =  torch.tensor(t_data)
    l_data = label_ll
    l_data = torch.tensor(l_data)
    for e in range(epoch):
        print('Started', e)
        losses = []
        #acc = []
        acc = None
        
        #train_data = next(iter(train_data)).type(torch.float32)
        #print('train_data')
        #print(train_data)
        #print(train_data[0, :-1])
        #print(train_data[0, :])
        #for i in range(5):
        #for rec in batch:
        #print(train_data)
        #features = train_data[:, :-1]#, :-1]
        features = t_data
        #print(features)
        #break
        #label = train_data[:, -1:]
        label = l_data
        print(label)
        #break
        optimizer.zero_grad()

        predicted = model(features)
        #print('predited')
        #print('p: ', len(predicted))
        #print('l: ', label)
        predicted = predicted.view(len(predicted), 1)
        label = label.view(len(label), 1)
        #print('pridicted size: ', predicted.shape)
        #print('label dType: ', label.shape)
        #print('pridicted: ', predicted)
        #print('label: ', label)
        loss_error = loss(predicted.double(), label.double())
        losses.append(loss_error.item())
        loss_error.backward()
        optimizer.step()
        #acc.append(accuracy(predicted.view(-1), label.view(-1)))
        acc = accuracy(predicted.view(-1), label.view(-1))
        #print('Average error at current  ', sum(losses)/len(losses))
        #print('done')
        #print(predicted)
        print('Average error at ',e,' epoch:  ', sum(losses)/len(losses))
        #acr = sum(acc)/len(acc)
        acr = acc
        print('Accuracy: ', acr)
        accuracy_count.append(acr)
        sum_loss.append(sum(losses)/len(losses))
        if acr >= 0.999:
            plt.plot(sum_loss, label = 'loss')
            #print(sum_loss)
            plt.legend()
            plt.show()
            return predicted, label
    #print('\nOverall Accuracy: ', sum(accuracy_count)/len(accuracy_count))
    #print('\n\n\n\n')
    #plt.plot(label, label = 'True')
    #plt.plot(sum_loss, label = 'loss')
    #print(sum_loss)
    #plt.legend()
    #plt.show()
    print('end of train')
    print("")
    return predicted, label


In [70]:
#train_dfs = pd.merge(features, label, left_index=True, right_index=True)

In [71]:
#train_dfs

In [72]:
#label = pd.DataFrame(train['Label'])

In [73]:
#label

In [75]:
pr, lr = train(10, traindataset, model, optimizer, loss)

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6020873208 bytes.

In [65]:
indices = torch.LongTensor([traindataset.row, traindataset.col])
values = torch.FloatTensor(traindataset.data)
shape = torch.Size(traindataset.shape)
tensor = torch.sparse_coo_tensor(indices, values, shape)

AttributeError: row not found

In [67]:
d = traindataset.toarray()

In [68]:
d

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
train_dfs.values

In [41]:
train['Label']

0       0
1       1
2       0
3       0
4       1
       ..
1858    0
1859    0
1860    1
1861    0
1862    0
Name: Label, Length: 1863, dtype: int64

In [36]:
len(train['Label'])

1863

In [31]:
train

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 